In [32]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from pathlib import Path
from math import sqrt
import sys
from datetime import datetime

In [52]:
class TimeSeriesRequestPredictor30MinHybrid:
    def __init__(self, sequence_length=144, prediction_horizon=48, rare_threshold=50):
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon
        self.rare_threshold = rare_threshold
        self.scaler = StandardScaler()
        self.geo_encoder = LabelEncoder()
        self.model = None
        self.rf_models = {}
        self.slot_counts = {}
        self.slot_means = {}
        self.global_slot_means = {}

    # ======================
    # Preprocess
    # ======================
    def load_and_preprocess(self, df):
        if df.empty:
            for col in ["request_date","origin_geo_hash","time_slot","request_count"]:
                if col not in df.columns:
                    df[col] = pd.NA
            return df

        df["request_date"] = pd.to_datetime(df["request_date"], errors="coerce", dayfirst=True)

        # Extract slot
        def extract_slot(val):
            if pd.isna(val):
                return -1
            if isinstance(val, str) and "-" in val:
                try:
                    h, m = map(int, val.split("-")[0].strip().split(":"))
                    return h*2 + m//30
                except:
                    return -1
            return -1
        df["slot_30min"] = df["time_slot"].apply(extract_slot)
        df["is_missing_slot"] = (df["slot_30min"]==-1).astype(int)

        # Encode geo
        if df["origin_geo_hash"].notna().sum() > 0:
            df["geo_encoded"] = self.geo_encoder.fit_transform(df["origin_geo_hash"])
        else:
            df["geo_encoded"] = 0

        df["day_of_week"] = df["request_date"].dt.dayofweek.fillna(0).astype(int)
        df["is_weekend"] = (df["day_of_week"] >=5).astype(int)

        df = df.sort_values(["origin_geo_hash","request_date","slot_30min"])
        return df

    # ======================
    # Training
    # ======================
    def train(self, df=None, epochs=20, validation_split=0.2):
        if df is None or df.empty:
            print("⚠️ Tidak ada data untuk train.")
            return df, None

        # Hitung slot counts & mean historis
        counts = df.groupby(["origin_geo_hash","slot_30min"]).size()
        self.slot_counts = counts.to_dict()
        self.slot_means = df.groupby(["origin_geo_hash","slot_30min"])["request_count"].mean().to_dict()
        self.global_slot_means = df.groupby("slot_30min")["request_count"].mean().to_dict()

        # Buat sequences
        X, y = self.create_sequences(df)
        if X.size != 0:
            # Split train/val
            split_index = int(len(X)*(1-validation_split))
            X_train, X_val = X[:split_index], X[split_index:]
            y_train, y_val = y[:split_index], y[split_index:]

            # Scale hanya di train
            self.scaler.fit(X_train.reshape(-1, X_train.shape[-1]))
            X_train_scaled = self.scaler.transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
            X_val_scaled   = self.scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

            # Build LSTM
            self.model = self.build_model((X.shape[1], X.shape[2]))
            callbacks = [
                EarlyStopping(patience=10, restore_best_weights=True),
                ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
            ]
            self.model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                           epochs=epochs, batch_size=32, callbacks=callbacks, verbose=1)
        else:
            print("⚠️ Tidak cukup data untuk LSTM.")

        # Random Forest untuk rare slots
        for (geo, slot), cnt in self.slot_counts.items():
            if cnt < self.rare_threshold:
                slot_df = df[(df["origin_geo_hash"]==geo) & (df["slot_30min"]==slot)]
                if len(slot_df) >= 3:
                    X_rf = slot_df[["day_of_week","is_weekend","geo_encoded"]].fillna(0)
                    y_rf = slot_df["request_count"].fillna(0)
                    rf = RandomForestRegressor(n_estimators=50, random_state=42)
                    try:
                        rf.fit(X_rf, y_rf)
                        self.rf_models[(geo, slot)] = rf
                    except Exception as e:
                        print(f"⚠️ RF fit error geo={geo}, slot={slot}: {e}")

        print(f"🔹 RF models trained: {len(self.rf_models)}")
        return df, None

    # ======================
    # Sequence generator
    # ======================
    def create_sequences(self, data):
        if data.empty:
            return np.array([]), np.array([])
        sequences, targets = [], []
        features = ["request_count","slot_30min","day_of_week","is_weekend","geo_encoded"]
        for geo in data["origin_geo_hash"].unique():
            geo_data = data[data["origin_geo_hash"]==geo].sort_values(["request_date","slot_30min"])
            feature_data = geo_data[features].fillna(0).values.copy()
            feature_data[:,0] = np.log1p(feature_data[:,0] + 1e-6)  # log-transform count
            for i in range(len(feature_data)-self.sequence_length-self.prediction_horizon+1):
                seq = feature_data[i:i+self.sequence_length]
                target = feature_data[i+self.sequence_length:i+self.sequence_length+self.prediction_horizon,0]
                sequences.append(seq)
                targets.append(target)
        if len(sequences)==0:
            return np.array([]), np.array([])
        return np.array(sequences), np.array(targets)

    # ======================
    # Build LSTM
    # ======================
    def build_model(self, input_shape):
        model = Sequential([
            Input(shape=input_shape),
            LSTM(128, return_sequences=True, recurrent_dropout=0.2),
            LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
            LSTM(32, dropout=0.2, recurrent_dropout=0.2),
            Dense(64, activation="relu"),
            Dropout(0.3),
            Dense(32, activation="relu"),
            Dropout(0.2),
            Dense(self.prediction_horizon, activation="linear")
        ])
        model.compile(optimizer=Adam(0.001), loss="mse", metrics=["mae"])
        return model

    # ======================
    # Prediction
    # ======================
    def predict_all_next_day_30min_filtered(self, df_last_day):
        if df_last_day.empty:
            return pd.DataFrame(columns=["geo_hash","date","slot_30min","predicted_request_count"])

        results = []
        for geo in df_last_day["origin_geo_hash"].unique():
            geo_data = df_last_day[df_last_day["origin_geo_hash"]==geo].sort_values("slot_30min")
            last_date = geo_data["request_date"].max()
            tomorrow_date = last_date + timedelta(days=1)
            all_slots = np.arange(48)

            out = {"geo_hash":[],"date":[],"slot_30min":[],"predicted_request_count":[]}
            for slot in all_slots:
                key = (geo,int(slot))
                if key in self.rf_models:   # RF model khusus
                    X_new = pd.DataFrame({
                        "day_of_week":[tomorrow_date.dayofweek],
                        "is_weekend":[int(tomorrow_date.dayofweek>=5)],
                        "geo_encoded":[self.geo_encoder.transform([geo])[0]]
                    })
                    try:
                        yhat = float(self.rf_models[key].predict(X_new)[0])
                    except:
                        yhat = self.slot_means.get((geo, slot),
                                                   self.global_slot_means.get(slot, 0.0))
                else:
                    # Fallback: pakai mean historis geo-slot > global-slot > 0
                    yhat = self.slot_means.get((geo, slot),
                                               self.global_slot_means.get(slot, 0.0))
                out["geo_hash"].append(geo)
                out["date"].append(tomorrow_date)
                out["slot_30min"].append(slot)
                out["predicted_request_count"].append(max(0,yhat))
            results.append(pd.DataFrame(out))
        return pd.concat(results, ignore_index=True)

# ======================
# Merge histori + new geos
# ======================
    def load_and_preprocess_with_new_geos(self, historical_df, new_df):
        df_hist = historical_df.copy() if historical_df is not None else pd.DataFrame()
        df_new = new_df.copy() if new_df is not None else pd.DataFrame()

        # pastikan kolom ada
        for df in [df_hist, df_new]:
            for col in ["request_date","origin_geo_hash","time_slot","request_count"]:
                if col not in df.columns:
                    df[col] = pd.NA

        # semua geo unik
        all_geos = pd.concat([
            df_hist.get("origin_geo_hash", pd.Series(dtype=str)),
            df_new.get("origin_geo_hash", pd.Series(dtype=str))
        ]).dropna().unique()

        last_date_hist = df_hist["request_date"].max() if not df_hist.empty else pd.Timestamp.today()

        # tambahkan row geo baru
        new_rows = []
        for geo in all_geos:
            if geo not in df_hist["origin_geo_hash"].unique():
                for slot in range(48):
                    new_rows.append({
                        "origin_geo_hash": geo,
                        "request_date": last_date_hist,
                        "time_slot": f"{slot//2:02d}:{(slot%2)*30:02d}-{slot//2:02d}:{(slot%2+1)*30:02d}",
                        "request_count": 0
                    })
        if new_rows:
            df_hist = pd.concat([df_hist, pd.DataFrame(new_rows)], ignore_index=True)

        df_full = pd.concat([df_hist, df_new], ignore_index=True) if not df_new.empty else df_hist
        return self.load_and_preprocess(df_full)
# ======================
# Fill missing slots untuk tanggal terakhir
# ======================
    @staticmethod
    def fill_missing_slots(df_last_day):
        all_slots = np.arange(48)
        all_geos = df_last_day["origin_geo_hash"].unique()
        new_rows = []
        for geo in all_geos:
            geo_slots = df_last_day[df_last_day["origin_geo_hash"]==geo]["slot_30min"].unique()
            missing_slots = set(all_slots) - set(geo_slots)
            for slot in missing_slots:
                new_rows.append({
                    "origin_geo_hash": geo,
                    "request_date": df_last_day["request_date"].max(),
                    "slot_30min": slot,
                    "request_count": 0
                })
        if new_rows:
            df_last_day = pd.concat([df_last_day, pd.DataFrame(new_rows)], ignore_index=True)
        return df_last_day

In [54]:
historical_file = "data_historis.csv"
file_new_data = "data_29_09_2025.csv"

# Load histori & data baru
historical_df = pd.read_csv(historical_file) if Path(historical_file).exists() else pd.DataFrame()
new_df = pd.read_csv(file_new_data) if Path(file_new_data).exists() else pd.DataFrame()

# Pastikan kolom selalu ada
for df in [historical_df, new_df]:
    for col in ["request_date", "origin_geo_hash", "time_slot", "request_count"]:
        if col not in df.columns:
            df[col] = pd.NA
    if not df.empty:
        df["request_date"] = pd.to_datetime(df["request_date"], errors="coerce", dayfirst=True)

# Gabungkan data baru ke histori
if not new_df.empty:
    historical_df = pd.concat([historical_df, new_df], ignore_index=True)
    historical_df.to_csv(historical_file, index=False)
    print(f"✅ Data baru ditambahkan. Total rows: {len(historical_df)}")
else:
    print("⚠️ File data baru tidak ditemukan. Menggunakan histori yang ada saja.")

# Guard: kalau tidak ada data sama sekali → stop
if historical_df.empty:
    print("❌ Tidak ada data untuk diproses.")
    sys.exit()

# Inisialisasi predictor
predictor = TimeSeriesRequestPredictor30MinHybrid()

# Preprocess data
df_full = predictor.load_and_preprocess_with_new_geos(historical_df, new_df)

# Training model
train_data, _ = predictor.train(df=df_full, epochs=20)

# Ambil tanggal terakhir
if df_full["request_date"].notna().any():
    max_date = pd.to_datetime(df_full["request_date"]).max()
else:
    print("❌ Tidak ada tanggal valid di dataset.")
    sys.exit()

# Ambil data terakhir & isi slot kosong
df_last_day = df_full[df_full["request_date"] == max_date].copy()
df_last_day = predictor.fill_missing_slots(df_last_day)

# Prediksi semua geo untuk H+1
df_pred = predictor.predict_all_next_day_30min_filtered(df_last_day)
df_pred["date"] = pd.to_datetime(df_pred["date"]).dt.strftime("%d-%m-%Y")

# Simpan prediksi dengan nama unik (hindari overwrite)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
prediksi_file = f"prediksi_{max_date.strftime('%d_%m_%Y')}_{timestamp}.csv"
df_pred.to_csv(prediksi_file, index=False)
print(f"✅ Prediksi tersimpan di {prediksi_file}")

⚠️ File data baru tidak ditemukan. Menggunakan histori yang ada saja.
Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 217ms/step - loss: 1.2725 - mae: 0.9581 - val_loss: 1.2690 - val_mae: 0.9522 - learning_rate: 0.0010
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - loss: 1.0084 - mae: 0.8273 - val_loss: 0.9230 - val_mae: 0.7796 - learning_rate: 0.0010
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - loss: 0.7808 - mae: 0.7093 - val_loss: 0.6646 - val_mae: 0.6341 - learning_rate: 0.0010
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - loss: 0.6474 - mae: 0.6337 - val_loss: 0.5561 - val_mae: 0.5555 - learning_rate: 0.0010
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 0.5640 - mae: 0.5875 - val_loss: 0.5184 - val_mae: 0.5349 - learning_rate: 0.0010
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 0.5277 - mae: 0.5642 - val_loss: 0.4814 - val_mae: 0.4989 - learning_rate: 0.0010
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 0.4941 - mae: 0.54

In [55]:
def evaluate_prediction(pred_file: str, actual_file: str, output_file="evaluasi_geo_hash_sama.csv"):

    # --- Baca file prediksi ---
    if not Path(pred_file).exists():
        print(f"⚠️ File {pred_file} tidak ditemukan.")
        return
    df_pred = pd.read_csv(pred_file)
    df_pred["date"] = pd.to_datetime(df_pred["date"], dayfirst=True).dt.date

    # --- Baca file actual ---
    if not Path(actual_file).exists():
        print(f"⚠️ File {actual_file} tidak ditemukan.")
        return
    df_actual = pd.read_csv(actual_file, skipinitialspace=True)
    df_actual.columns = df_actual.columns.str.strip()

    if "request_date" not in df_actual.columns:
        print("⚠️ Kolom 'request_date' tidak ditemukan di CSV actual!")
        return

    # Convert tanggal
    df_actual["request_date"] = pd.to_datetime(df_actual["request_date"], dayfirst=True)

    # Extract slot
    def extract_slot(val):
        try:
            h, m = map(int, val.split("-")[0].strip().split(":"))
            return h * 2 + (m // 30)
        except:
            return -1

    df_actual["slot_30min"] = df_actual["time_slot"].apply(extract_slot)
    df_actual["geo_hash"] = df_actual["origin_geo_hash"]
    df_actual["date"] = df_actual["request_date"].dt.date
    df_actual["actual_request_count"] = df_actual["request_count"]

    # Pilih kolom penting
    df_actual_proc = df_actual[["geo_hash","date","slot_30min","actual_request_count"]]

    # Filter hanya geo_hash yang ada di prediksi
    valid_geo = df_pred["geo_hash"].unique()
    df_actual_proc = df_actual_proc[df_actual_proc["geo_hash"].isin(valid_geo)]

    # Merge inner
    merged = pd.merge(
        df_pred,
        df_actual_proc,
        on=["geo_hash","date","slot_30min"],
        how="inner"
    )

    if not merged.empty:
        mae = mean_absolute_error(merged["actual_request_count"], merged["predicted_request_count"])
        r2  = r2_score(merged["actual_request_count"], merged["predicted_request_count"])
        rmse = sqrt(mean_squared_error(merged["actual_request_count"], merged["predicted_request_count"]))
        print(f"📊 Evaluasi (geo_hash sama): MAE={mae:.4f}, R²={r2:.4f}, RMSE={rmse:.4f}, N={len(merged)}")
    else:
        print("⚠️ Tidak ada slot/geo_hash yang cocok untuk evaluasi.")

    # Simpan hasil evaluasi
    merged.to_csv(output_file, index=False)
    print(f"✅ Hasil evaluasi disimpan di {output_file}")


In [56]:
evaluate_prediction(
    pred_file="prediksi_28_09_2025_20251001_141040.csv",
    actual_file="data_29_09_202.csv",
    output_file="evaluasi_manual.csv"
)

📊 Evaluasi (geo_hash sama): MAE=2.7525, R²=-0.3137, RMSE=4.0944, N=1083
✅ Hasil evaluasi disimpan di evaluasi_manual.csv


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import timedelta

class RandomForestPredictor:
    def __init__(self, history_file, today_file, use_today=False):
        self.history_file = history_file
        self.today_file = today_file
        self.use_today = use_today
        self.model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=1, verbose=1)

        # Encoders
        self.geohash_encoder = LabelEncoder()

    def _load_data(self, file_path):
        df = pd.read_csv(file_path)

        # Bersihkan nama kolom dari spasi & tanda kutip
        df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")

        # Rename kolom sesuai format internal
        df.rename(columns={
            "request_date": "date",
            "origin_geo_hash": "geohash",
            "time_slot": "time_slot",
            "request_count": "count"
        }, inplace=True)

        # Pastikan kolom wajib ada
        required = {"date", "time_slot", "geohash", "count"}
        missing = required - set(df.columns)
        if missing:
            raise ValueError(f"CSV {file_path} missing required columns: {missing}")

        # Parse date
        df["date"] = pd.to_datetime(df["date"], errors='coerce', dayfirst=True)

        # Hapus data dengan count 0
        if "count" in df.columns:
            df = df[df["count"] > 0]

        return df

    
    def _create_full_grid(self, df):
        # Comment to remove filling 0
        dates = df["date"].unique()
        timeslots = df["time_slot"].unique()
        geohashes = df["geohash"].unique()
        
        grid = pd.MultiIndex.from_product([dates, timeslots, geohashes], names=["date", "time_slot", "geohash"]).to_frame(index=False)

        df = pd.merge(grid, df, on=["date", "time_slot", "geohash"], how="left")
        df["count"] = df["count"].fillna(0)

        df.to_csv(self.history_file, index=False)
        return df

    def preprocess(self):
        # Load data
        history = self._load_data(self.history_file)
        today = self._load_data(self.today_file)

        # Combine history + today
        combined = pd.concat([history, today], ignore_index=True)

        # Fill missing combinations for the whole dataset
        combined = self._create_full_grid(combined)

        # Parse timeslot
        combined["slot_start"] = combined["time_slot"].str.split("-").str[0]
        combined["slot_start"] = pd.to_datetime(combined["slot_start"], format="%H:%M")
        combined["hour"] = combined["slot_start"].dt.hour
        combined["minute"] = combined["slot_start"].dt.minute
        combined.drop(columns=["slot_start"], inplace=True)

        # Encode geohash
        combined["geohash_encoded"] = self.geohash_encoder.fit_transform(combined["geohash"])

        # Extract date features
        combined["day"] = combined["date"].dt.day
        combined["month"] = combined["date"].dt.month
        combined["weekday"] = combined["date"].dt.weekday

        # Split back into history and today
        history = combined[combined["date"] < combined["date"].max()]
        today = combined[combined["date"] == combined["date"].max()]

        return combined, history["date"].max(), today["date"].max()
    
    def train_and_predict(self, target_date=None):
        combined, last_history_date, today_date = self.preprocess()

        # Tentukan tanggal target prediksi
        if target_date is None:
            # default: besok dari hari terakhir
            target_date = (today_date if self.use_today else last_history_date) + timedelta(days=1)
        else:
            target_date = pd.to_datetime(target_date)

        # Split kembali history vs today
        history = combined[combined["date"] <= last_history_date].copy()
        today = combined[combined["date"] == today_date].copy()

        if self.use_today:
            train = pd.concat([history, today], ignore_index=True)
        else:
            train = history

        # Features untuk training
        features = ["geohash_encoded", "hour", "minute", "day", "month", "weekday"]
        X_train = train[features]
        y_train = train["count"]

        # Train model
        self.model.fit(X_train, y_train)

        # Siapkan data prediksi untuk target_date
        if target_date == today_date:
            pred_df = today.copy()
        else:
            # Ambil data terakhir, ubah tanggal ke target_date
            pred_df = combined[combined["date"] == today_date].copy()
            pred_df["date"] = target_date

        X_pred = pred_df[features]
        pred_df["prediction"] = self.model.predict(X_pred)

        # Save hasil prediksi
        pred_file = f"forest_{target_date.strftime('%Y-%m-%d')}_prediction.csv"
        pred_df[["date", "time_slot", "geohash", "prediction"]].to_csv(pred_file, index=False)

        print(f"✅ Prediction saved to {pred_file}")
        return pred_file


    def evaluate_by_file(pred_file, actual_file, output_file="eval_manual.csv"):
        import pandas as pd
        from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
        import numpy as np

        # --- Load prediksi ---
        pred_df = pd.read_csv(pred_file)
        pred_df.rename(columns={"prediction": "count_pred"}, inplace=True)
        pred_df["date"] = pd.to_datetime(pred_df["date"], format="%Y-%m-%d").dt.date

        # --- Load actual ---
        actual_df = pd.read_csv(actual_file)
        actual_df.rename(columns={
            "request_date": "date",
            "origin_geo_hash": "geohash",
            "request_count": "count_actual"
        }, inplace=True)
        actual_df["date"] = pd.to_datetime(actual_df["date"], dayfirst=True).dt.date

        # Merge
        eval_df = pred_df.merge(actual_df, on=["date", "time_slot", "geohash"], how="inner")

        if eval_df.empty:
            print("⚠ Tidak ada data cocok untuk evaluasi (cek date/time_slot/geohash).")
            return

        mae = mean_absolute_error(eval_df["count_actual"], eval_df["count_pred"])
        rmse = np.sqrt(mean_squared_error(eval_df["count_actual"], eval_df["count_pred"]))
        r2 = r2_score(eval_df["count_actual"], eval_df["count_pred"])

        print(f"📊 Evaluasi: MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.2f}, N={len(eval_df)}")

        eval_df.to_csv(output_file, index=False)
        print(f"✅ Hasil evaluasi disimpan di {output_file}")

In [ ]:
# --- Inisialisasi predictor ---
predictor = RandomForestPredictor(
    history_file="data_historis.csv",
    today_file="data_29_09_202.csv",
    use_today=False  # supaya training cuma sampai 28
)



In [103]:
# Prediksi 29-09-2025
pred_file = predictor.train_and_predict(target_date="2025-09-29")

# Evaluasi lawan data aktual
evaluate_by_file(
    pred_file=pred_file,
    actual_file="data_29_09_202.csv",
    output_file="eval_2025-09-29.csv"
)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    9.8s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   37.3s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   37.4s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


✅ Prediction saved to forest_2025-09-29_prediction.csv
✅ Hasil evaluasi disimpan di eval_2025-09-29.csv
📊 MAE=3.0862, RMSE=4.5956, R²=-0.7168, N=1163
